In [1]:
import pandas as pd
import requests
import json

C:\Users\faical\AppData\Local\Temp\ipykernel_6380\3527380040.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Inputs are required, so that the code will run as workflow in codelos without issues
input = dict()
results = dict()
exec_path = "."

In [3]:
# Define excel-path (will be replaced with s3-logic or similar later)
file_path = f"{exec_path}/20230830_RLM Power & Gas_Portfolio_.xlsx"

In [4]:
# Functions
def get_local_item(commodity:str):
    '''
    Returns a item, is not used in prod-environment later
    '''
    # Specify the path to your JSON file
    file_path = f"./{commodity}_item.json"

    # Open the JSON file and load its contents
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
        
    return data

def get_database_name(load_id: str):
    '''
    Returns the database-id within a load-id
    '''
    return load_id.split('/')[-1].split('::')[0]


def get_measure_type(item:object, commodity:str):
    '''
    Returns the correct string for the excel column "Messung"
    Either of: RLM/SLP, SLP/RLM, RLM, SLP
    '''
    item_key = 'lieferstellenuebersichtgas' if commodity == 'gas' else 'lieferstellenuebersicht'
    counter_types = [i['counterType'] for i in item[item_key]]
    unique_values = list(set(counter_types))

    return "/".join(unique_values)

def get_service_cost(item:object, commodity:str):
    '''
    Returns the correct string for the excel column "Messung"
    Either of: RLM/SLP, SLP/RLM, RLM, SLP
    '''
    item_key = 'lieferstellenuebersichtgas' if commodity == 'gas' else 'lieferstellenuebersicht'
    
    # Get prices
    service_billing_fees = {'RLM': None, 'SLP': None}
    for entry in item[item_key]:
        counter_type = entry['counterType']
        service_billing_fees[counter_type] = entry['serviceBillingFee']
    
    return service_billing_fees

def get_delivery_point_amount(item:object, commodity:str):
    '''
    Returns the correct string for the excel column "Messung"
    Either of: RLM/SLP, SLP/RLM, RLM, SLP
    '''
    item_key = 'lieferstellenuebersichtgas' if commodity == 'gas' else 'lieferstellenuebersicht'
    
    # Get prices
    delivery_point_amount = {'RLM': 0, 'SLP': 0}
    for entry in item[item_key]:
        counter_type = entry['counterType']
        delivery_point_amount[counter_type] += 1
    
    return delivery_point_amount

In [5]:
# Code

In [5]:
# Item will come like this in codelos
input['custom'] = get_local_item('gas') #'power'

In [6]:
# Dont work with the input var, its not good overview
item = input['custom']

In [7]:
# Check commodity
commodity = item['commodity'].lower()
sheet_name = 'Lastgänge_gas' if commodity == 'gas' else 'Lastgänge_Strom'

print('Commodity:', commodity)
print('Sheet:', sheet_name)

Commodity: gas
Sheet: Lastgänge_gas


In [8]:
# Get all items for Excel
import openpyxl

wb=openpyxl.load_workbook(file_path)
worksheet= wb[sheet_name]
# Excel power row: 33
company_name = item['kunde']
col_max= worksheet.max_column+1

cellref=worksheet.cell(row=33, column= col_max)

cellref.value=company_name

# Excel power row: 34
pfc_name = item['namepfc']

cellref=worksheet.cell(row=34, column= col_max)

cellref.value=pfc_name

# Excel row power: 36
measure_type = get_measure_type(item, commodity)

cellref=worksheet.cell(row=36, column= col_max)

cellref.value= measure_type
# Get delivery point amount per type 
# Excel power row: RLM: 37, SLP, 38
delivery_point_amount = get_delivery_point_amount(item, commodity)
if 'RLM' in delivery_point_amount.keys() : 
    cellref=worksheet.cell(row=37, column= col_max)
    cellref.value= delivery_point_amount['RLM']
if 'SLP' in delivery_point_amount.keys() : 
    cellref=worksheet.cell(row=38, column= col_max)
    cellref.value= delivery_point_amount['SLP']
# RLM row: 39, SLP row:40
service_cost = get_service_cost(item, commodity)
if 'RLM' in service_cost.keys() : 
    cellref=worksheet.cell(row=39, column= col_max)
    cellref.value= service_cost['RLM']
if 'SLP' in service_cost.keys() : 
    cellref=worksheet.cell(row=40, column= col_max)
    cellref.value= service_cost['SLP']

# Make labor price in €/MWh (power row: 42/43/44/45)
sum_kWh = item['sumkwh_yearly']
labor_price = item['arbeitspreisctkwh_yearly']
for i in range(len(sum_kWh)):
    cellref=worksheet.cell(row=42+i, column= col_max)
    x= sum_kWh[i]['value']/(labor_price[i]['value']*1000)
    cellref.value=round(x, 2)
# Calculate basic price as in excel (power row: 47, 48, 49, 50)
sum_MWh = item['summwh_yearly']
diff= 0 
for i in service_cost.keys():
   diff+= delivery_point_amount[i]*service_cost[i]

for i in range(len(sum_kWh)):
    cellref=worksheet.cell(row=47+i, column= col_max)
    
    x= sum_kWh[i]['value']/(labor_price[i]['value']*1000) + diff/sum_MWh[i]['value']

    cellref.value=round(x, 2)
# Sum mwh in excel (power row: 52, 53, 54, 55)
#sum_MWh = item['summwh_yearly']
for i in range(len(sum_MWh)):
    cellref=worksheet.cell(row=52+i, column= col_max)
    x = sum_MWh[i]['value']
    cellref.value=round(x, 2)
# Load is always in correct unit already (kW for power) (kWh for gas) (power row 56)

# Create link to codelos-db
# Excel power row: 35
load_id = item['pfc_loadprofile_id']
pfc_link = f"https://www.app.codelos.io/#/?section=data&database={get_database_name(load_id)}/item={load_id}"
cellref=worksheet.cell(row=35, column= col_max)

cellref.value=pfc_link
wb.save(file_path)

In [64]:
delivery_point_amount

{'RLM': 4, 'SLP': 1}

In [ ]:
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
# Here some logic has to be added
# We want to add date to the "new col" while starting in row 34 with the name of the company
# After that we must save it in the same excel, with the exact same structure as currently existing
# We don't need the formula, as we calculate all the values here (for that particular sheet)

,Std.,Monat,MW,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,744,2024-01-01 00:00:00,2.065559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,696,2024-02-01 00:00:00,2.028744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,744,2024-03-01 00:00:00,1.620005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,720,2024-04-01 00:00:00,0.997812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,744,2024-05-01 00:00:00,0.454479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1145,Dezember 2026,2026,5.06293,0,39.0953,197.936774,2026-12-27 06:00:00,NaN,1971.741039,3091.189121,NaN
1146,Dezember 2026,2026,5.107538,0,39.3095,200.774773,2026-12-28 06:00:00,NaN,1989.211949,3118.326253,NaN
1147,Dezember 2026,2026,4.983856,0,39.3701,196.2149,2026-12-29 06:00:00,NaN,1941.028643,3042.827122,NaN
1148,Dezember 2026,2026,4.904243,0,39.4992,193.713694,2026-12-30 06:00:00,NaN,1910.039384,2994.204086,NaN
